In [3]:
# 標準使用ライブラリー
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
# Suppress warnings 
import warnings
warnings.filterwarnings('ignore')
import gc
import os
import shutil

# matplotlib and seaborn for plotting
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns

import json
import datetime
import math
plt.style.use('dark_background')

# debug
#%pdb on

import pixiedust #%pixie_debugger

In [38]:
from pathlib import Path
import ffmpeg

In [39]:
MOVIE_DIR = Path("/home/input/openpose_tryout/movies/")
MOVIE2IMG_DIR = Path( "/home/input/openpose_tryout/movies_output/")


In [ ]:
def prior_convert(file):
    '''mov, mp4 を jpg, mp3に分離する'''
    mp4_output_path = file.parent / "{}.mp4".format(file.stem)
    mp3_output_path = file.parent / "{}.mp3".format(file.stem)
    jpg_output_path = SDANCE_CACHE_INPUT_IMAGE_DIR / "%06d.jpg"
    SDANCE_CACHE_INPUT_IMAGE_DIR
    # mp4が存在しなければmovをmp4に変換する
    if not mp4_output_path.is_file():
        ffmpeg.input(str(file)) \
        .output(filename=str(mp4_output_path), pix_fmt='yuv420p') \
        .run()
    # mp4から音声mp3を抽出する
    if mp3_output_path.is_file():
        mp3_output_path.unlink()
    ffmpeg.input(str(mp4_output_path)) \
    .output(filename=str(mp3_output_path), map='0:1') \
    .run()
    # mp4から画像jpgを抽出する
    for f in SDANCE_CACHE_INPUT_IMAGE_DIR.glob("*.jpg"):
        f.unlink()
    ffmpeg.input(str(mp4_output_path)) \
    .output(filename=str(jpg_output_path), ss=0, r=30000/1001, start_number=000000) \
    .run()

In [64]:
stream = ffmpeg.input(str(MOVIE_DIR) + "/moriwaki_360.mp4")

In [63]:
str(MOVIE_DIR) + "moriwaki_360.mp4"

'/home/input/openpose_tryout/moviesmoriwaki_360.mp4'

In [56]:
# 画像を空にする
for f in MOVIE2IMG_DIR.glob("*.jpg"):
    f.unlink()

In [65]:
stream.output(
    filename=str(MOVIE2IMG_DIR / "%06d.jpg"), ss=0, r=30000/1001, start_number=000000)\
    .run()
#     filename= str("/home/input/openpose_tryout/movies_output/")+"%06d.jpg", ss=0, r=30000/1001, start_number=000000)\
#     filename= "/home/input/openpose_tryout/movies/"+"%06d.jpg", ss=0, r=30000/1001, start_number=000000)\
#     filename= "/home/input/openpose_tryout/movies/1.jpg", ss=0, r=30000/1001) \


(None, None)

In [51]:
stream.output(filename=str("/home/input/openpose_tryout/movies_output/"+ "%06d.jpg"), ss=0, r=30000/1001, start_number=000000).run()

Error: ffmpeg error (see stderr output for detail)